# NonlinearModel
* Uses LightGBM to estimate a given (user, item) prediction

In [ ]:
task = "temporal_causal"
medium = "anime"
training_frac = 0.9

In [ ]:
import NBInclude: @nbinclude
@nbinclude("TreeModelBase.ipynb")

In [ ]:
train_model(
    [
        explicit_raw_alphas(task, medium)
        implicit_raw_alphas(task, medium)
        nondirectional_raw_alphas(medium)
        "$medium/$task/LinearExplicit"
        "$medium/$task/LinearImplicit"
    ],
    String["$medium/$task/LinearExplicit"],
    task,
    ["explicit"],
    medium,
    false,
    "test",
    training_frac,
    "$medium/$task/NonlinearExplicit";
    λ = nothing,
);

In [ ]:
train_model(
    [
        explicit_raw_alphas(task, medium)
        implicit_raw_alphas(task, medium)
        nondirectional_raw_alphas(medium)
        "$medium/$task/LinearExplicit"
        "$medium/$task/LinearImplicit"
    ],
    String[],
    task,
    ["implicit", "negative"],
    medium,
    true,
    "test",
    training_frac,
    "$medium/$task/NonlinearImplicit";
    λ = nothing,
);

In [ ]:
function save_final_explicit_model()
    function model(
        split::String,
        task::String,
        content::String,
        medium::String;
        raw_splits = true,
    )
        split_fn = raw_splits ? get_raw_split : get_split
        alpha_fn = raw_splits ? read_raw_alpha : read_alpha
        if split == "training"
            return zeros(Float32, length(split_fn(split, task, content, medium).user))
        end
        linear_split =
            alpha_fn("$medium/$task/LinearExplicit", split, task, content, medium).rating
        nonlinear_split =
            alpha_fn("$medium/$task/NonlinearExplicit", split, task, content, medium).rating
        linear_split + nonlinear_split
    end
    write_params(
        Dict(
            "alphas" => ["$medium/$task/LinearExplicit", "$medium/$task/NonlinearExplicit"],
        ),
        "$task/Explicit",
    )
    write_alpha(
        model,
        medium,
        "$medium/$task/Explicit";
        task = task,
        by_split = true,
        log = true,
        log_alphas = String[],
        log_task = task,
        log_content = "explicit",
        log_splits = ["test"],
    )
end;

In [ ]:
save_final_explicit_model()